In [93]:
import pandas as pd 
import numpy as np 
from ast import literal_eval
from tabulate import tabulate  # Import tabulate for clean table formatting
import itertools


In [3]:
movies = pd.read_parquet("tmbd_exports/tmbd_5000_api-call_merged.parquet")

In [5]:
movies.columns

Index(['budget', 'genres', 'homepage', 'movie_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'director'],
      dtype='object')

In [7]:
movies.head(5)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,cast,director
0,237000000,"Action, Adventure, Fantasy, Science Fiction",http://www.avatarmovie.com/,19995,"culture clash, future, space war, space colony...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",James Cameron
1,300000000,"Adventure, Fantasy, Action",http://disney.go.com/disneypictures/pirates/,285,"ocean, drug abuse, exotic island, east india t...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"Johnny Depp, Orlando Bloom, Keira Knightley, S...",Gore Verbinski
2,245000000,"Action, Adventure, Crime",http://www.sonypictures.com/movies/spectre/,206647,"spy, based on novel, secret agent, sequel, mi6",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,880674609,148.0,Released,A Plan No One Escapes,Spectre,6.3,4466,"Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...",Sam Mendes
3,250000000,"Action, Crime, Drama, Thriller",http://www.thedarkknightrises.com/,49026,"dc comics, crime fighter, terrorist, secret id...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.0,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"Christian Bale, Michael Caine, Gary Oldman, An...",Christopher Nolan
4,260000000,"Action, Adventure, Science Fiction",http://movies.disney.com/john-carter,49529,"based on novel, mars, medallion, space travel,...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,284139100,132.0,Released,"Lost in our world, found in another.",John Carter,6.1,2124,"Taylor Kitsch, Lynn Collins, Samantha Morton, ...",Andrew Stanton


* STEP 1: TURN STRING COLUMNS INTO USEABLE OBJECTS

In [11]:
# List of columns that need fixing
columns_to_fix = ['cast', 'genres', 'director','keywords']

# Convert raw comma-separated text into properly formatted lists (Vectorized)
movies[columns_to_fix] = movies[columns_to_fix].apply(lambda col: 
    col.str.split(', ').apply(lambda x: str(x) if isinstance(x, list) else "[]")
)

# Apply literal_eval to convert them into actual lists
movies[columns_to_fix] = movies[columns_to_fix].applymap(literal_eval)

C:\Users\sara\AppData\Local\Temp\ipykernel_13432\3398161126.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  movies[columns_to_fix] = movies[columns_to_fix].applymap(literal_eval)


In [13]:
#check
movies[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes],"[spy, based on novel, secret agent, sequel, mi6]","[Action, Adventure, Crime]"


In [16]:
#return the list top 3 elements or entire list, whichever is more. This makes the handling
def get_list(x):
    if isinstance(x, list):
        # Check if the list contains dictionaries or just plain strings -> fixes potential TypeError:string indices must be integers, not 'str'
        if all(isinstance(i, dict) for i in x):  
            names = [i['name'] for i in x if 'name' in i]  # Extract 'name' if it's a dictionary
        else:
            names = x  # If it's a list of strings, use it directly
        
        # Return top 3 elements or the entire list
        return names[:3] if len(names) > 3 else names
    
    return []  # Return an empty list if data is missing/malformed

# Define the features to apply the function
features = ['cast', 'keywords', 'genres']

for feature in features:
    movies[feature] = movies[feature].apply(get_list)

In [18]:
#check
movies[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


* STEP 2: CREATE FILTER BASED ON GENRE USER INPUT, CALCULATE WEIGHTED RATING OF SUBSET AND RETURN TOP 10 MOVIES

In [56]:
# Function to get all unique genres in the dataset -> this allows to provide a list of available genres before user input
def get_all_genres(movies_df):
    unique_genres = set(genre for sublist in movies_df['genres'] for genre in sublist)
    return sorted(unique_genres)  # Sort for better readability

# Function to filter movies by genre (case-insensitive) -> without this if user wrote 'action' instead of 'Action', no movies would come up 
def filter_movies_by_genre(movies_df, selected_genre):
    selected_genre = selected_genre.lower()  # Convert user input to lowercase
    return movies_df[movies_df['genres'].apply(lambda x: any(g.lower() == selected_genre for g in x))]

# Function to calculate weighted rating based on vote count threshold
def weighted_rating_threshold(data_frame, qntl=0.9):  
    if not {"vote_average", "vote_count"}.issubset(data_frame.columns):
        raise ValueError("The columns 'vote_average' and 'vote_count' are missing in the DataFrame!")

    m = data_frame["vote_count"].quantile(qntl)  # Minimum votes required
    C = data_frame["vote_average"].mean()  # Mean vote across all movies

    # Filter movies that meet the vote threshold
    q_movies = data_frame.loc[data_frame["vote_count"] >= m].copy()

    # Compute weighted rating
    def weighted_rating(q_movs, m=m, C=C):
        R = q_movs["vote_average"]
        v = q_movs["vote_count"]
        return (v/(v+m) * R) + (m/(m+v) * C)

    q_movies['score'] = q_movies.apply(weighted_rating, axis=1)    

    return q_movies

# Display available genres
available_genres = get_all_genres(movies)
print("\nAvailable Genres:\n" + ", ".join(available_genres) + "\n")

#  Get user input
user_genre = input("Enter a genre from the list above: ").strip()

# Step 2: Filter dataset based on input genre
filtered_movies = filter_movies_by_genre(movies, user_genre)

#  Apply weighted rating function to the filtered dataset
if not filtered_movies.empty:
    ranked_movies = weighted_rating_threshold(filtered_movies)
    # Step 4: Sort by weighted score and return top 10
    top_movies = ranked_movies.sort_values(by="score", ascending=False).head(10)

    print("\nTop 10 Movies in the Selected Genre (Ranked by Weighted Rating):\n")
    print(tabulate(top_movies[['title', 'score', 'vote_average', 'vote_count']], 
                   headers="keys", tablefmt="pretty", showindex=False))  # Pretty table format

else:
    print("\nNo movies found in this genre.")



Available Genres:
, Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, Foreign, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, Western



Enter a genre from the list above:  crime



Top 10 Movies in the Selected Genre (Ranked by Weighted Rating):

+--------------------------+-------------------+--------------+------------+
|          title           |       score       | vote_average | vote_count |
+--------------------------+-------------------+--------------+------------+
| The Shawshank Redemption | 8.105386424659319 |     8.5      |    8205    |
|     The Dark Knight      | 7.951885814641981 |     8.2      |   12002    |
|       Pulp Fiction       | 7.947542532243347 |     8.3      |    8428    |
|      The Godfather       | 7.907447140389091 |     8.4      |    5893    |
|          Se7en           | 7.667107822837041 |     8.1      |    5765    |
|      The Green Mile      | 7.608154280462569 |     8.2      |    4048    |
|  The Godfather: Part II  | 7.590980042502643 |     8.3      |    3338    |
| The Silence of the Lambs | 7.573847367487092 |     8.1      |    4443    |
| The Wolf of Wall Street  | 7.550306977620614 |     7.9      |    6571    |
|        

* STEP 2.B: FILTER WITH DIRECTOR
* Caveat of this is that for now you need to enter full name of director

In [69]:
# Function to get all unique directors in the dataset -> this allows to provide a list of available genres before user input
def get_all_directors(movies_df):
    unique_directors = set(director for sublist in movies_df['director'] for director in sublist)
    return sorted(unique_directors)  # Sort for better readability

# Function to filter movies by genre (case-insensitive) -> without this if user wrote 'action' instead of 'Action', no movies would come up 
def filter_movies_by_director(movies_df, selected_director):
    selected_director = selected_director.lower()  # Convert user input to lowercase
    return movies_df[movies_df['director'].apply(lambda x: any(g.lower() == selected_director for g in x))]

# Function to calculate weighted rating based on vote count threshold
def weighted_rating_threshold(data_frame, qntl=0.9):  
    if not {"vote_average", "vote_count"}.issubset(data_frame.columns):
        raise ValueError("The columns 'vote_average' and 'vote_count' are missing in the DataFrame!")

    m = data_frame["vote_count"].quantile(qntl)  # Minimum votes required
    C = data_frame["vote_average"].mean()  # Mean vote across all movies

    # Filter movies that meet the vote threshold
    q_movies = data_frame.loc[data_frame["vote_count"] >= m].copy()

    # Compute weighted rating
    def weighted_rating(q_movs, m=m, C=C):
        R = q_movs["vote_average"]
        v = q_movs["vote_count"]
        return (v/(v+m) * R) + (m/(m+v) * C)

    q_movies['score'] = q_movies.apply(weighted_rating, axis=1)    

    return q_movies

# Display available genres
available_director = get_all_directors(movies)
print("\nAvailable Directors:\n" + ", ".join(available_director) + "\n")

#  Get user input
user_director = input("Enter a director: ").strip()

# Step 2: Filter dataset based on input genre
filtered_movies = filter_movies_by_director(movies, user_director)

#  Apply weighted rating function to the filtered dataset
if not filtered_movies.empty:
    ranked_movies = weighted_rating_threshold(filtered_movies)
    # Step 4: Sort by weighted score and return top 10
    top_movies = ranked_movies.sort_values(by="score", ascending=False).head(10)

    print("\nTop 10 Movies from selected Director (Ranked by Weighted Rating):\n")
    print(tabulate(top_movies[['title', 'score', 'vote_average', 'vote_count']], 
                   headers="keys", tablefmt="pretty", showindex=False))  # Pretty table format

else:
    print("\nNo movies found for this director.")



Available Directors:
A. Raven Cruz, Aaron Hann, Aaron Schneider, Aaron Seltzer, Aaron T. Wells, Abel Ferrara, Adam Brooks, Adam Carolla, Adam Goldberg, Adam Green, Adam Jay Epstein, Adam Marcus, Adam McKay, Adam Montierth, Adam Rapp, Adam Rifkin, Adam Shankman, Adrian Lyne, Adrienne Shelly, Agnieszka Holland, Agnieszka Wojtowicz-Vosloo, Agustín Díaz Yanes, Aki Kaurismäki, Akira Kurosawa, Akiva Goldsman, Akiva Schaffer, Al Franklin, Al Silliman Jr., Alain Resnais, Alan Alda, Alan Cohn, Alan J. Pakula, Alan Metter, Alan Parker, Alan Poul, Alan Rudolph, Alan Shapiro, Alan Smithee, Alan Taylor, Alan Yuen, Albert Brooks, Albert Hughes, Alejandro Agresti, Alejandro Amenábar, Alejandro González Iñárritu, Alejandro Monteverde, Aleksandr Veledinskiy, Aleksei German, Aleksey German, Alex Cox, Alex Craig Mann, Alex Garland, Alex Gibney, Alex Kendrick, Alex Proyas, Alex Ranarivelo, Alex Rivera, Alex Smith, Alex Zamm, Alex van Warmerdam, Alexander Payne, Alexander Witt, Alexandr Veledinsky, Alexan

Enter a director:  epstein



No movies found for this director.


* STEP 2C: FILTER BY GENRE AND DIRECTOR

In [105]:
# Function to get all unique genres in the dataset
def get_all_genres(movies_df):
    unique_genres = set(genre for sublist in movies_df['genres'] for genre in sublist)
    return sorted(unique_genres)  # Sort for better readability

# Function to get all directors for a selected genre
def get_directors_for_genre(movies_df, selected_genre):
    # Filter movies by genre (case-insensitive)
    genre_filtered = movies_df[movies_df['genres'].apply(lambda x: selected_genre.lower() in [g.lower() for g in x])]
    
    # Flatten the lists of directors and get unique directors
    directors_flattened = list(itertools.chain(*genre_filtered['director']))  # Flatten the list of directors
    return sorted(set(directors_flattened))  # Return unique directors

# Function to filter movies by genre (case-insensitive)
def filter_movies_by_genre(movies_df, selected_genre):
    selected_genre = selected_genre.lower()  # Convert user input to lowercase
    return movies_df[movies_df['genres'].apply(lambda x: any(g.lower() == selected_genre for g in x))]

# Function to filter movies by director (case-insensitive)
def filter_movies_by_director(movies_df, selected_director):
    selected_director = selected_director.lower()  # Convert user input to lowercase
    return movies_df[movies_df['director'].apply(lambda directors: any(d.lower() == selected_director for d in directors))]

# Function to calculate weighted rating based on vote count threshold
def weighted_rating_threshold(data_frame, qntl=0.9):  
    if not {"vote_average", "vote_count"}.issubset(data_frame.columns):
        raise ValueError("The columns 'vote_average' and 'vote_count' are missing in the DataFrame!")

    m = data_frame["vote_count"].quantile(qntl)  # Minimum votes required
    C = data_frame["vote_average"].mean()  # Mean vote across all movies

    # Filter movies that meet the vote threshold
    q_movies = data_frame.loc[data_frame["vote_count"] >= m].copy()

    # Compute weighted rating
    def weighted_rating(q_movs, m=m, C=C):
        R = q_movs["vote_average"]
        v = q_movs["vote_count"]
        return (v/(v+m) * R) + (m/(m+v) * C)

    q_movies['score'] = q_movies.apply(weighted_rating, axis=1)    

    return q_movies

# Step 0: Display available genres
available_genres = get_all_genres(movies)
print("\nAvailable Genres:\n" + ", ".join(available_genres) + "\n")

# Step 1: Get user input for genre
user_genre = input("Enter a genre from the list above: ").strip()

# Step 2: Filter dataset based on selected genre
filtered_movies_by_genre = filter_movies_by_genre(movies, user_genre)

# Step 3: Get a list of directors for the selected genre
available_directors = get_directors_for_genre(filtered_movies_by_genre, user_genre)

if available_directors:
    print("\nAvailable Directors in this Genre:\n" + ", ".join(available_directors) + "\n")
else:
    print(f"\nNo directors found for the genre '{user_genre}'.")
    exit()

# Step 4: Get user input for director (ensure the director is from the available list)
user_director = input(f"Enter a director from the list above: ").strip()

# Step 5: Filter dataset by both genre and director
filtered_movies_by_genre_and_director = filter_movies_by_director(filtered_movies_by_genre, user_director)

# Step 6: Apply weighted rating function to the filtered dataset
if not filtered_movies_by_genre_and_director.empty:
    ranked_movies = weighted_rating_threshold(filtered_movies_by_genre_and_director)
    # Step 7: Sort by weighted score and return top 10
    top_movies = ranked_movies.sort_values(by="score", ascending=False).head(10)

    print("\nTop 10 Movies in the Selected Genre and Director (Ranked by Weighted Rating):\n")
    print(tabulate(top_movies[['title', 'score', 'vote_average', 'vote_count']], 
                   headers="keys", tablefmt="pretty", showindex=False))  # Pretty table format

else:
    print("\nNo movies found in this genre and director combination.")


Available Genres:
, Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, Foreign, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, Western



Enter a genre from the list above:  comedy



Available Directors in this Genre:
A. Raven Cruz, Aaron Schneider, Aaron Seltzer, Aaron T. Wells, Adam Brooks, Adam Carolla, Adam Green, Adam Jay Epstein, Adam McKay, Adam Rapp, Adam Rifkin, Adam Shankman, Adrienne Shelly, Aki Kaurismäki, Akira Kurosawa, Akiva Schaffer, Al Silliman Jr., Alan Alda, Alan Cohn, Alan Metter, Alan Poul, Alan Rudolph, Alan Smithee, Albert Brooks, Alejandro González Iñárritu, Alejandro Monteverde, Alex Cox, Alex Craig Mann, Alex Zamm, Alex van Warmerdam, Alexander Payne, Alexandre Aja, Alfred Hitchcock, Alice Wu, Alison Maclean, Allan Arkush, Allison Anders, Amy Heckerling, Analeine Cal y Mayor, Anand Tucker, Andrei Konchalovsky, Andrew Adamson, Andrew Bergman, Andrew Bujalski, Andrew Currie, Andrew Fleming, Andrew Gurland, Andrew Jacobson, Andrew Stanton, Andrew Wilson, Andrés Couturier, Andy Cadiff, Andy Fickman, Andy Tennant, Ang Lee, Angela Robinson, Anna Mastro, Annabel Jankel, Anne Fletcher, Anthony Bell, Anthony Russo, Anthony Stacchi, Antonia Bird, A

Enter a director from the list above:  wayne wang



Top 10 Movies in the Selected Genre and Director (Ranked by Weighted Rating):

+-------------------+-------------------+--------------+------------+
|       title       |       score       | vote_average | vote_count |
+-------------------+-------------------+--------------+------------+
| Maid in Manhattan | 5.774471053243432 |     5.6      |    485     |
+-------------------+-------------------+--------------+------------+
